# 도서 정보 객체

In [1]:
class Book:
    def __init__(self, rank, title, author, price):
        self.rank = rank
        self.title = title
        self.author = author
        self.price = price

    def __str__(self):
        return f"{self.rank}, {self.title}, {self.author}, {self.price}"
    
    def to_dict(self):
        return {'rank':self.rank, 
                'title':self.title, 
                'author':self.author, 
                'price':self.price}
    
    def to_list(self):
        return [self.rank, 
                self.title, 
                self.author, 
                self.price]

#yesBestList > li:nth-child(1) > div > div.item_info

In [2]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
best_list_el = soup.select('#yesBestList div.item_info')
len(best_list_el)


24

In [3]:
best_list_el[0]

<div class="item_info">
<div class="info_row info_keynote">
<span class="gd_keynote" id="spanGdKeynote">
<a class="iconC tBook" href="/24/Category/More/001014?ElemNo=13&amp;ElemSeq=1&amp;Gcode=000_030_001"><em class="txt">오늘의책</em></a>
<!-- 클래스24 상품일 경우 -->
</span>
<script type="text/javascript">
        if ($('#spanGdKeynote').children().length == 0) {
            $('#spanGdKeynote').remove();
        }
    </script>
</div>
<div class="info_row info_name">
<span class="gd_res">[도서]</span>
<a class="gd_name" href="/product/goods/143911524" onclick="wiseLogV2('BS', '001_005_001', ''); ">혼모노</a>
<a class="bgYUI ico_nWin" href="/product/goods/143911524" onclick="wiseLogV2('BS', '001_005_001', ''); " target="_blank">혼모노 새창이동</a>
</div>
<div class="info_row info_pubGrp">
<span class="authPub info_auth" onclick="wiseLogV2('BS', '001_005_002', '');">
<a href="https://www.yes24.com/product/search?domain=ALL&amp;query=%25EC%2584%25B1%25ED%2595%25B4%25EB%2582%2598&amp;authorNo=253719&amp;author=

In [4]:
book_list = []
for i, item in enumerate(best_list_el):
  title = item.select_one('div.info_name > a').text
  author = item.select_one('.info_auth > a').text
  price = item.select_one('div.info_price .txt_num').text
  book_list.append(Book(i+1,title, author, price))
  
for book in book_list:
  print(book)

1, 혼모노, 성해나, 16,200원
2, 류수영의 평생 레시피, 류수영, 22,500원
3, 가공범, 히가시노 게이고, 19,800원
4, 돌비공포라디오 더 레드, 돌비, 17,820원
5, 2025 큰별쌤 최태성의 별별한국사 기출 500제 한국사능력검정시험 심화(1,2,3급), 최태성, 17,550원
6, 료의 생각 없는 생각, 료, 18,000원
7, 박곰희 연금 부자 수업, 박곰희, 18,900원
8, 2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 상, 최태성, 14,850원
9, 단 한 줄만 내 마음에 새긴다고 해도, 나민애, 21,420원
10, 자몽살구클럽, 한로로, 10,800원
11, ETS 토익 정기시험 기출문제집 1000 Vol. 4 RC, ETS, 17,820원
12, 2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 하, 최태성, 14,400원
13, ETS 토익 정기시험 기출문제집 1000 Vol. 4 LC, ETS, 17,820원
14, 안녕이라 그랬어, 김애란, 15,120원
15, 다크 심리학, 다크 사이드 프로젝트, 19,710원
16, 모순, 양귀자, 11,700원
17, 편안함의 습격, 마이클 이스터, 19,800원
18, 어른의 행복은 조용하다, 태수, 16,020원
19, 견우와 선녀 대본집 세트, 양지훈, 41,400원
20, 청춘의 독서, 유시민, 17,010원
21, 야구선수 김원중, 김원중, 18,000원
22, 소년이 온다, 한강, 13,500원
23, 첫 여름, 완주, 김금희, 15,300원
24, 경험의 멸종, 크리스틴 로젠, 17,820원


In [5]:
query = '''
CREATE TABLE IF NOT EXISTS BOOKS(
  RANK INTEGER PRIMARY KEY,
  TITLE TEXT,
  AUTHOR TEXT,
  PRICE INTEGER
)
'''

In [6]:
import sqlite3
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()
cursor.execute(query)
conn.commit()
conn.close()

In [7]:
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

In [8]:
ins_query = '''
INSERT INTO books (rank, title, author, price) VALUES (?,?,?,?)
'''

for book in book_list:
  cursor.execute(ins_query, book.to_list())
  
conn.commit()
conn.close()

# db table에서 읽어오기

In [9]:
def fetch_book_list_from_db():
  conn = sqlite3.connect('my_database.db')
  cursor= conn.cursor()
  try:
    cursor.execute('select * from books')
    rows = cursor.fetchall()
    return rows
  finally:
    conn.close()  

In [10]:
books = fetch_book_list_from_db()
for book in books:
  print(book)

(1, '혼모노', '성해나', '16,200원')
(2, '류수영의 평생 레시피', '류수영', '22,500원')
(3, '가공범', '히가시노 게이고', '19,800원')
(4, '돌비공포라디오 더 레드', '돌비', '17,820원')
(5, '2025 큰별쌤 최태성의 별별한국사 기출 500제 한국사능력검정시험 심화(1,2,3급)', '최태성', '17,550원')
(6, '료의 생각 없는 생각', '료', '18,000원')
(7, '박곰희 연금 부자 수업', '박곰희', '18,900원')
(8, '2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 상', '최태성', '14,850원')
(9, '단 한 줄만 내 마음에 새긴다고 해도', '나민애', '21,420원')
(10, '자몽살구클럽', '한로로', '10,800원')
(11, 'ETS 토익 정기시험 기출문제집 1000 Vol. 4 RC', 'ETS', '17,820원')
(12, '2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 하', '최태성', '14,400원')
(13, 'ETS 토익 정기시험 기출문제집 1000 Vol. 4 LC', 'ETS', '17,820원')
(14, '안녕이라 그랬어', '김애란', '15,120원')
(15, '다크 심리학', '다크 사이드 프로젝트', '19,710원')
(16, '모순', '양귀자', '11,700원')
(17, '편안함의 습격', '마이클 이스터', '19,800원')
(18, '어른의 행복은 조용하다', '태수', '16,020원')
(19, '견우와 선녀 대본집 세트', '양지훈', '41,400원')
(20, '청춘의 독서', '유시민', '17,010원')
(21, '야구선수 김원중', '김원중', '18,000원')
(22, '소년이 온다', '한강', '13,500원')
(23, '첫 여름, 완주', '김금희', '15,300원')
(24, '경험의 멸종', '크리스틴 로젠', 

# 여러페이지 가져오기

In [19]:
base_url = 'https://www.yes24.com/product/category/bestseller?categoryNumber=001&pageSize=24&pageNumber='
page_no = 3
book_list = []

for page in range(1, page_no+1):
    # 페이지 연결
    url = f'{base_url}{page}'
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    best_list_el = soup.select('#yesBestList div.item_info')
    
    # 요소 추출
    for i, item in enumerate(best_list_el):
        title = item.select_one('.info_name > a').text
        author = item.select_one(".info_auth > a").text
        price_el = item.select_one(".info_price .txt_num")
        # price 대체하기
        price_text = price_el.text.strip().replace(",", "").replace("원", "") if price_el else "0"
        price = int(price_text)
        
        rank = (page - 1) * len(best_list_el) + (i + 1)  # 고유 rank 부여
        book_list.append(Book(rank ,title, author, price))
len(book_list)

72

In [20]:
for book in book_list:
    print(book)

1, 혼모노, 성해나, 16200
2, 류수영의 평생 레시피, 류수영, 22500
3, 가공범, 히가시노 게이고, 19800
4, 돌비공포라디오 더 레드, 돌비, 17820
5, 2025 큰별쌤 최태성의 별별한국사 기출 500제 한국사능력검정시험 심화(1,2,3급), 최태성, 17550
6, 료의 생각 없는 생각, 료, 18000
7, 박곰희 연금 부자 수업, 박곰희, 18900
8, 2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 상, 최태성, 14850
9, 단 한 줄만 내 마음에 새긴다고 해도, 나민애, 21420
10, 자몽살구클럽, 한로로, 10800
11, ETS 토익 정기시험 기출문제집 1000 Vol. 4 RC, ETS, 17820
12, 2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 하, 최태성, 14400
13, ETS 토익 정기시험 기출문제집 1000 Vol. 4 LC, ETS, 17820
14, 안녕이라 그랬어, 김애란, 15120
15, 다크 심리학, 다크 사이드 프로젝트, 19710
16, 모순, 양귀자, 11700
17, 편안함의 습격, 마이클 이스터, 19800
18, 어른의 행복은 조용하다, 태수, 16020
19, 견우와 선녀 대본집 세트, 양지훈, 41400
20, 청춘의 독서, 유시민, 17010
21, 야구선수 김원중, 김원중, 18000
22, 소년이 온다, 한강, 13500
23, 첫 여름, 완주, 김금희, 15300
24, 경험의 멸종, 크리스틴 로젠, 17820
25, 인생을 바꾸는 최고의 ETF, 잼투리, 22050
26, 푸른 사자 와니니 8, 이현, 12420
27, 어른의 품격을 채우는 100일 필사 노트, 김종원, 18000
28, 해커스 토익 기출 VOCA (보카), David Cho, 11610
29, 궤도, 서맨사 하비, 15300
30, 2026 유휘운 행정법총론 요약노트+기출문제 (요.플.), 유휘운, 31500
31, 러닝&마라톤 무작정 따라하기

In [22]:
import pymysql

# DB 연결
con = pymysql.connect(host='localhost', user='root', password='1234', db='스크랩핑실습', charset='utf8')
cursor = con.cursor()

# 테이블 생성
create_table_sql = ''' 
CREATE TABLE IF NOT EXISTS BOOKS (
    `RANK` INTEGER PRIMARY KEY,
    TITLE TEXT,
    AUTHOR TEXT, 
    PRICE INTEGER
)
'''
cursor.execute(create_table_sql)
con.commit()

In [23]:
insert_sql = '''
INSERT INTO BOOKS (`rank`, title, author, price)
VALUES (%s, %s, %s, %s)
'''
for book in book_list:
    cursor.execute(insert_sql, book.to_list())

con.commit()